In [1]:
import pickle
import boris_extraction as boris
import multirecording_spikeanalysis as spike
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.stats import sem
from itertools import combinations
def hex_2_rgb(hex_color): # Orange color
    rgb_color = tuple(int(hex_color[i:i+2], 16) / 255.0 for i in (1, 3, 5))
    return rgb_color

def pickle_this(thing_to_pickle, file_name):
    """
    Pickles things
    Args (2):   
        thing_to_pickle: anything you want to pickle
        file_name: str, filename that ends with .pkl 
    Returns:
        none
    """
    with open(file_name,'wb') as file:
        pickle.dump(thing_to_pickle, file)

def unpickle_this(pickle_file):
    """
    Unpickles things
    Args (1):   
        file_name: str, pickle filename that already exists and ends with .pkl
    Returns:
        pickled item
    """
    with open(pickle_file, 'rb') as file:
        return(pickle.load(file))

## Wilcoxon Single Cell Analysis

In [2]:
phase3_collection = unpickle_this('phase3collection.pkl')
phase3_analysis = spike.SpikeAnalysis_MultiRecording(phase3_collection, timebin = 100, ignore_freq = 0.5)

All set to analyze


In [3]:
           #offset included                    offset excluded 
# 2, 5, -2 =      80                                 72
# 1 , 5, -2 =     65                                  65
# 0, 6, -1 =      65                                       
# 1, 5, -1 =      64                                  58
# 2, 4, 1 =       62
# 3, 5, 0 =       62
# 3, 5, -1 =      61
# 3, 5, -2 =      61                                  
# 2, 4, -1 =      71                                  73
# 2, 4, 0 =       59         
# 3, 4, 0 =       59

In [4]:
equalize = 3
baseline_window = 1
offset = 0
print(phase3_analysis.fishers_exact_wilcox('novel', 'cagemate', equalize = equalize, baseline_window=baseline_window,offset= offset, event3 = None))
phase3_analysis.fishers_exact_wilcox('familiar', 'acquisition',equalize = equalize, baseline_window=baseline_window,offset= offset, event3 = None)

Wilcoxon can't be done on 20230809_103121_1_merged.rec novel, because <6 samples
Wilcoxon can't be done on 20230809_143303_1_merged.rec cagemate, because <6 samples
(0.6622516556291391, 0.2732220442656473, [[15, 151], [18, 120]])
Wilcoxon can't be done on 20230809_103121_1_merged.rec familiar, because <6 samples
Wilcoxon can't be done on 20230810_140909_1_merged.rec familiar, because <6 samples


(1.6766355140186915, 0.1349124403847154, [[23, 107], [20, 156]])

In [5]:
for event in ['novel', 'cagemate', 'familiar', 'acquisition']:
    phase3_analysis.wilcox_baseline_v_event_collection(event=event, equalize =equalize, baseline_window=baseline_window, offset=offset, plot = False, save = True)

Wilcoxon can't be done on 20230809_103121_1_merged.rec novel, because <6 samples
Wilcoxon can't be done on 20230809_143303_1_merged.rec cagemate, because <6 samples
Wilcoxon can't be done on 20230809_103121_1_merged.rec familiar, because <6 samples
Wilcoxon can't be done on 20230810_140909_1_merged.rec familiar, because <6 samples


In [6]:
df_list = []
for comparison in ['4s novel vs 2s baseline', '4s cagemate vs 2s baseline', '4s familiar vs 2s baseline', '4s acquisition vs 2s baseline']:
    df = phase3_collection.wilcox_dfs[comparison]
    df_list.append(df)

is_first = True 
for df in df_list:
    if is_first:
        master_df = df[['Subject', 'Recording', 'original unit id', 
                            'Event','event1 vs event2']]
        is_first = False
    else: 
        temp_df = df[['Subject', 'Recording', 'original unit id', 
                            'Event', 'event1 vs event2']]
        master_df = master_df.merge(temp_df,  on=['Subject', 'Recording', 'original unit id'], how = 'left')
    
#novel vs baseline

KeyError: '4s novel vs 2s baseline'

In [ ]:
master_df.columns = ['Subject',
 'Recording',
 'original unit id',
 'Event_x',
 'novel vs baseline',
 'Event_y',
 'cagemate vs baseline',
 'Event_x',
 'familiar vs baseline',
 'Event_y',
 'acquisition vs baseline']
master_df = master_df[['original unit id',
 'novel vs baseline',
 'cagemate vs baseline',
 'familiar vs baseline',
 'acquisition vs baseline']]
master_df

,original unit id,novel vs baseline,cagemate vs baseline,familiar vs baseline,acquisition vs baseline
0,0,NaN,NaN,NaN,NaN
1,45,not significant,NaN,not significant,increases
2,289,not significant,NaN,increases,not significant
3,279,increases,NaN,not significant,not significant
4,293,not significant,NaN,not significant,increases
...,...,...,...,...,...
162,76,not significant,not significant,not significant,not significant
163,149,increases,not significant,not significant,not significant
164,135,not significant,not significant,not significant,not significant
165,38,increases,increases,not significant,not significant


In [ ]:
def group_cells(novel, cagemate, fam, acquisition):
    group = ''
    if (novel == 'increases') or (novel == 'decreases'):
        group = 'novel'
    if (cagemate == 'increases') or (cagemate == 'decreases'):
        group = group + 'cagemate' 
    if (fam == 'increases') or (fam == 'decreases'):
        group = group + 'fam'
    if (acquisition == 'increases') or (acquisition == 'decreases'):
        group = group + 'acquisition'
    return group

master_df['cell group'] = master_df.apply(lambda row: group_cells(row['novel vs baseline'],
                                                                  row['cagemate vs baseline'],
                                                                  row['familiar vs baseline'],
                                                                  row['acquisition vs baseline']), axis = 1)
results = master_df.groupby('cell group').count()
results

C:\Users\megha\AppData\Local\Temp\ipykernel_19616\2096176807.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  master_df['cell group'] = master_df.apply(lambda row: group_cells(row['novel vs baseline'],


,original unit id,novel vs baseline,cagemate vs baseline,familiar vs baseline,acquisition vs baseline
cell group,,,,,
,104,103,91,70,103
acquisition,12,12,8,12,12
cagemate,8,8,8,7,8
cagemateacquisition,5,5,5,5,5
cagematefam,3,3,3,3,3
fam,10,10,4,10,10
famacquisition,8,8,0,8,8
novel,9,9,5,7,9
novelcagemate,2,2,2,2,2


In [ ]:
percents = results['original unit id'] / results['original unit id'].sum()
percents

cell group
                       0.622754
acquisition            0.095808
cagemate               0.047904
cagemateacquisition    0.017964
fam                    0.047904
famacquisition         0.041916
novel                  0.077844
novelcagemate          0.011976
novelcagematefam       0.005988
novelfam               0.017964
novelfamacquisition    0.011976
Name: original unit id, dtype: float64

In [ ]:
results['original unit id'].sum()

147

In [7]:
df_list = []
for comparison in ['3s novel vs 1s baseline', '3s cagemate vs 1s baseline', '3s familiar vs 1s baseline', '3s acquisition vs 1s baseline']:
    df = phase3_collection.wilcox_dfs[comparison]
    df_list.append(df)

is_first = True 
for df in df_list:
    if is_first:
        master_df = df[['Subject', 'Recording', 'original unit id', 
                            'Event','event1 vs event2']]
        is_first = False
    else: 
        temp_df = df[['Subject', 'Recording', 'original unit id', 
                            'Event', 'event1 vs event2']]
        master_df = master_df.merge(temp_df,  on=['Subject', 'Recording', 'original unit id'], how = 'left')
master_df.columns = ['Subject',
 'Recording',
 'original unit id',
 'Event_x',
 'novel vs baseline',
 'Event_y',
 'cagemate vs baseline',
 'Event_x',
 'familiar vs baseline',
 'Event_y',
 'acquisition vs baseline']
master_df = master_df[['original unit id',
 'novel vs baseline',
 'cagemate vs baseline',
 'familiar vs baseline',
 'acquisition vs baseline']]
master_df   

def group_cells(novel, cagemate, fam, acquisition):
    group = ''
    if (novel == 'increases') or (novel == 'decreases'):
        group = 'novel'
    if (cagemate == 'increases') or (cagemate == 'decreases'):
        group = group + 'cagemate' 
    if (fam == 'increases') or (fam == 'decreases'):
        group = group + 'fam'
    if (acquisition == 'increases') or (acquisition == 'decreases'):
        group = group + 'acquisitiimpoon'
    return group

master_df['cell group'] = master_df.apply(lambda row: group_cells(row['novel vs baseline'],
                                                                  row['cagemate vs baseline'],
                                                                  row['familiar vs baseline'],
                                                                  row['acquisition vs baseline']), axis = 1)
results = master_df.groupby('cell group').count()
results

C:\Users\megha\AppData\Local\Temp\ipykernel_28324\3260278753.py:15: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Event_x', 'event1 vs event2_x'} in the result is deprecated and will raise a MergeError in a future version.
  master_df = master_df.merge(temp_df,  on=['Subject', 'Recording', 'original unit id'], how = 'left')


,original unit id,novel vs baseline,cagemate vs baseline,familiar vs baseline,acquisition vs baseline
cell group,,,,,
,108,107,96,75,107
acquisition,11,11,3,10,11
cagemate,13,13,13,12,13
cagemateacquisition,2,2,2,2,2
cagematefam,1,1,1,1,1
fam,12,12,4,12,12
famacquisition,5,5,1,5,5
novel,9,9,6,7,9
novelacquisition,1,1,1,1,1


In [8]:
percents = results['original unit id'] / results['original unit id'].sum()
percents

cell group
                       0.646707
acquisition            0.065868
cagemate               0.077844
cagemateacquisition    0.011976
cagematefam            0.005988
fam                    0.071856
famacquisition         0.029940
novel                  0.053892
novelacquisition       0.005988
novelfam               0.029940
Name: original unit id, dtype: float64

In [10]:
equalize = 3
baseline_window = 1
offset = 0
for event in ['cup 1', 'cup 2', 'cup 3', 'cup 4']:
    phase3_analysis.wilcox_baseline_v_event_collection(event=event, equalize =equalize, baseline_window=baseline_window, offset=offset, plot = False, save = True)

Wilcoxon can't be done on 20230809_103121_1_merged.rec cup 1, because <6 samples
Wilcoxon can't be done on 20230809_143303_1_merged.rec cup 1, because <6 samples
Wilcoxon can't be done on 20230810_101534_1_merged.rec cup 1 275, because baseline = event
Wilcoxon can't be done on 20230810_140909_1_merged.rec cup 1, because <6 samples
Wilcoxon can't be done on 20230809_103121_1_merged.rec cup 2, because <6 samples
Wilcoxon can't be done on 20230810_120856_1_merged.rec cup 2, because <6 samples
Wilcoxon can't be done on 20230811_101635_1_merged.rec cup 2, because <6 samples
Wilcoxon can't be done on 20230810_140909_1_merged.rec cup 3, because <6 samples
Wilcoxon can't be done on 20230809_103121_1_merged.rec cup 4, because <6 samples
Wilcoxon can't be done on 20230809_143303_1_merged.rec cup 4, because <6 samples


In [13]:
df_list = []
for comparison in ['3s cup 1 vs 1s baseline', '3s cup 2 vs 1s baseline', '3s cup 3 vs 1s baseline', '3s cup 4 vs 1s baseline']:
    df = phase3_collection.wilcox_dfs[comparison]
    df_list.append(df)

is_first = True 
for df in df_list:
    if is_first:
        master_df = df[['Subject', 'Recording', 'original unit id', 
                            'Event','event1 vs event2']]
        is_first = False
    else: 
        temp_df = df[['Subject', 'Recording', 'original unit id', 
                            'Event', 'event1 vs event2']]
        master_df = master_df.merge(temp_df,  on=['Subject', 'Recording', 'original unit id'], how = 'left')

master_df.columns = ['Subject',
 'Recording',
 'original unit id',
 'Event_x',
 'cup 1 vs baseline',
 'Event_y',
 'cup 2 vs baseline',
 'Event_x',
 'cup 3 vs baseline',
 'Event_y',
 'cup 4 vs baseline']
master_df = master_df[['original unit id',
 'cup 1 vs baseline',
 'cup 2 vs baseline',
 'cup 3 vs baseline',
 'cup 4 vs baseline']]
master_df

def group_cells(cup_1, cup_2, cup_3, cup_4):
    group = ''
    if (cup_1 == 'increases') or (cup_1 == 'decreases'):
        group = 'cup 1'
    if (cup_2 == 'increases') or (cup_2 == 'decreases'):
        group = group + 'cup_2' 
    if (cup_3 == 'increases') or (cup_3 == 'decreases'):
        group = group + 'cup_3'
    if (cup_4 == 'increases') or (cup_4 == 'decreases'):
        group = group + 'cup_4'
    return group

master_df['cell group'] = master_df.apply(lambda row: group_cells(row['cup 1 vs baseline'],
                                                                  row['cup 2 vs baseline'],
                                                                  row['cup 3 vs baseline'],
                                                                  row['cup 4 vs baseline']), axis = 1)
results = master_df.groupby('cell group').count()
results

C:\Users\megha\AppData\Local\Temp\ipykernel_28324\240993788.py:15: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Event_x', 'event1 vs event2_x'} in the result is deprecated and will raise a MergeError in a future version.
  master_df = master_df.merge(temp_df,  on=['Subject', 'Recording', 'original unit id'], how = 'left')


,original unit id,cup 1 vs baseline,cup 2 vs baseline,cup 3 vs baseline,cup 4 vs baseline
cell group,,,,,
,67,64,43,64,64
cup 1,5,5,2,5,5
cup 1cup_4,1,1,0,1,1
cup_2,3,3,3,3,3
cup_3,6,6,4,6,6
cup_3cup_4,1,1,0,1,1
cup_4,12,12,11,12,12
